In [1]:
import pandas as pd
import cobra
import logging

In [2]:
def change_medium(model, sub_type, sub):
    '''
        {'r_1714'}; % D-glucose exchange
        {'r_1654'}; % ammonium exchange
        {'r_2005'}; % phosphate exchange
        {'r_2060'}; % phosphate exchange
    '''
    model_test = model.copy()
    medium = model_test.medium
    medium[sub.id] = 10.0
    if sub_type == 'C':
        medium['r_1714'] = 0.0
    elif sub_type == 'N':
        medium['r_1654'] = 0.0
    elif sub_type == 'P':
        medium['r_2005'] = 0.0
    elif sub_type == 'S':
        medium['r_2060'] = 0.0
    model_test.medium = medium
    return model_test

In [3]:
def add_exchange(model, metid):
    model_test = model.copy()
    try:
        rxn = model_test.add_boundary(model_test.metabolites.get_by_id(metid))
        logging.info('add {}'.format(rxn.reaction))
    except ValueError:
        met = model_test.metabolites.get_by_id(metid)
        print(metid)
        met.compartment = 'e'
        model_test.add_metabolites(met)
        rxn = model_test.add_boundary(met, type="exchange")
        logging.info('add {}'.format(rxn.reaction))
    return model_test, rxn

In [4]:
def simu(model):
    sol = model.optimize()
    if sol.objective_value > 1e-6:
        gng = 'G'
    else:
        gng = 'NG'
    return gng


logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(filename)s[line:%(lineno)d] - %(levelname)s: %(message)s')
biolog = pd.read_table('./Biolog_Substrate.tsv')

In [5]:
model = cobra.io.read_sbml_model('./yeast-GEM.xml')
exp_simu = pd.DataFrame(columns=['Name_in_Model', 'id', 'Substrate_type', 'Growth_Biolog', 'Growth_Model'])
for m in model.metabolites:
    for i in range(len(biolog.index)):
        if m.name == biolog.loc[i, 'Name_in_Model']:
        #if m.name.split(' ')[0] == biolog.loc[i, 'Name_in_Model']:
            exp_simu.loc[i, 'id'] = m.id
            exp_simu.loc[i, 'Substrate_type'] = biolog.loc[i, 'Substrate_type']
            exp_simu.loc[i, 'Growth_Biolog'] = biolog.loc[i, 'Growth_Biolog']
            exp_simu.loc[i, 'Name_in_Model'] = biolog.loc[i, 'Name_in_Model']

exp_simu.index = range(len(exp_simu))


if __name__ == '__main__':
    for i in range(len(exp_simu.index)):
        model_test, rxn = add_exchange(model, exp_simu.loc[i, 'id'])
        model_final = change_medium(model_test, exp_simu.loc[i, 'Substrate_type'], rxn)
        gng = simu(model_final)
        logging.info('{}: {}'.format(str(exp_simu.loc[i, 'Name_in_Model']), gng))
        exp_simu.loc[i, 'Growth_Model'] = gng

    c = 0
    for i in range(len(exp_simu.index)):
        if exp_simu.loc[i, 'Growth_Biolog'] == exp_simu.loc[i, 'Growth_Model']:
            c += 1

    exp_simu.to_excel('./yeast9_sub_use.xlsx'.format(model.id))

2024-03-06 19:42:27,812 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-03-06 19:42:27,813 - sbml.py[line:1672] - DEBUG: Unexpected content format 'Saccharomyces cerevisiae - strain S288C'.
2024-03-06 19:42:34,190 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-03-06 19:42:34,750 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-03-06 19:42:34,757 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-03-06 19:42:34,758 - 3936220034.py[line:12] - INFO: add s_0065 <=> 
2024-03-06 19:42:34,759 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.


s_0065


2024-03-06 19:42:35,228 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-03-06 19:42:35,242 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-03-06 19:42:35,778 - 1263953134.py[line:20] - INFO: (S)-lactate: G
2024-03-06 19:42:35,780 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-03-06 19:42:36,153 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-03-06 19:42:36,158 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-03-06 19:42:36,159 - 3936220034.py[line:12] - INFO: add s_0069 <=> 
2024-03-06 19:42:36,160 - model.py[line:209] - INFO: T

s_0069


2024-03-06 19:42:36,664 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-03-06 19:42:36,678 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-03-06 19:42:37,198 - 1263953134.py[line:20] - INFO: (S)-malate: G
2024-03-06 19:42:37,199 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-03-06 19:42:37,703 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-03-06 19:42:37,704 - 3936220034.py[line:5] - INFO: add s_4161 <=> 
2024-03-06 19:42:37,705 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-03-06 19:42:38,066 - boundary_types.py[line:68] - INFO: Compartment `e` sounds l

...

2024-03-06 19:47:34,372 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-03-06 19:47:34,387 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-03-06 19:47:34,950 - 1263953134.py[line:20] - INFO: Gly-Met: G
2024-03-06 19:47:34,952 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-03-06 19:47:35,381 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-03-06 19:47:35,386 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-03-06 19:47:35,387 - 3936220034.py[line:12] - INFO: add s_4087 <=> 
2024-03-06 19:47:35,389 - model.py[line:209] - INFO: The c

s_4087


2024-03-06 19:47:35,858 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-03-06 19:47:35,873 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-03-06 19:47:36,408 - 1263953134.py[line:20] - INFO: Gly-Met: G
2024-03-06 19:47:36,409 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-03-06 19:47:36,845 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-03-06 19:47:36,846 - 3936220034.py[line:5] - INFO: add s_4090 <=> 
2024-03-06 19:47:36,847 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-03-06 19:47:37,263 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like

s_4121


2024-03-06 19:47:59,894 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-03-06 19:47:59,908 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-03-06 19:48:00,425 - 1263953134.py[line:20] - INFO: Ala-Asp: G
2024-03-06 19:48:00,426 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-03-06 19:48:00,860 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like an external compartment. Using this one without counting boundary reactions.
2024-03-06 19:48:00,860 - 3936220034.py[line:5] - INFO: add s_4125 <=> 
2024-03-06 19:48:00,860 - model.py[line:209] - INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
2024-03-06 19:48:01,289 - boundary_types.py[line:68] - INFO: Compartment `e` sounds like